In [1]:
import PyPDF2
import regex as re

In [2]:
# Request user to input the filename without the .pdf
# filename = input('What is the file name?\n')
filename = 'D6'

# Enter the sections to be extracted here
section_to_extract = [18,6,7,8,9,12,13,14,15,16,17,20,21,22,23,24,25,27]
section_to_extract.sort()

## Getting the PDF File Text

This section of code will handle obtaining the file and the text within that pdf file.

In [3]:
# Open the File
file = open(f'data/{filename}.pdf','rb')

# Create PyPDF2 Reader Object
reader = PyPDF2.PdfFileReader(file)

# Get the Text from the file
text = ''

for i in range(reader.numPages):
    # Get the current page
    page = reader.getPage(i)
    # Extract the text
    current_text = page.extract_text()
    # Append to master text variable
    text += current_text

## Separate the text by Section

This section of code will separate the text by section and return a dictionary of section and the corresponding text.

In [4]:
# Section Regex
section_regex = re.escape(filename) + r'\.\d{1,2}\n'

# Create a Regex to Identify section numbers
sections_full = re.findall(section_regex,text)

# Split the text into an array
section_text = re.split(section_regex,text)

# Get a list of Sections
sections = [s.strip('\n') for s in sections_full]

# Remove the first section (Before D#.1)
section_text.pop(0)

# Create Dictionary
doc_dict = dict(zip(sections,section_text))

## Extract Target Data

This section of code will search for the sections specified by the user and return it as a string.

In [5]:
def remove_linespace_OLD(text):
    # Split by .\n to get an array
    # linespace_regex = r'(?<=[a-z)])\.\n(?=[A-Z(])'
    linespace_regex = r'[\.;:]\n'
    
    delimiter_arr = re.findall(linespace_regex,text)
    text_arr = re.split(linespace_regex,text)
    
    # Strip '\n'
    text_arr_formatted = [s.replace('\n','') for s in text_arr]
    
    # Replace '\n' with '\r\n'
    delimiter_arr_formatted = [s.replace('\n','') for s in delimiter_arr]
    
    # Join together
    full_text_arr = []
    
    for idx, text_formatted in enumerate(text_arr_formatted):
        if idx == len(text_arr_formatted) - 1:
            
            # Check if the section starts with "End of Document"
            if text_formatted.find("End of Document") == -1:
                # End of Document not Found
                full_text = text_formatted
                
        elif idx > 0:
            # Start implementing delimiter + text
            full_text = delimiter_arr_formatted[idx-1]
            full_text += text_formatted
        
        else:
            # First section of the text won't have a delimiter
            full_text += text_formatted
        
        full_text_arr.append(full_text)

        
    return full_text

In [6]:
def remove_linespace(text_raw) -> list:
    
    linespace_regex = r'[\.;:]\n'
    
    delimiter_arr = re.findall(linespace_regex,text_raw)
    text_arr = re.split(linespace_regex,text_raw)
    
    # Strip '\n'
    text_arr_formatted = [s.replace('\n','') for s in text_arr]
    
    # Replace '\n' with '\r\n'
    delimiter_arr_formatted = [s.replace('\n','') for s in delimiter_arr]
    
    text_delim_zip = zip(text_arr_formatted,delimiter_arr_formatted)
    
    text_arr = []
    
    for text, delimiter in text_delim_zip:
        text_arr.append(text + delimiter)
    
    return text_arr

## Generating Word File

This section will start creating the word file based on the text obtained and sections highlighted.

In [7]:
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_BREAK

In [8]:
doc = Document()

In [9]:
# Create Heading for Document
doc.add_heading(f"{filename} Compilation",0)

In [10]:
# Add Sections and Text to Document
for target in section_to_extract:
    section_heading = f'{filename}.{target}'
    section_text = remove_linespace(doc_dict[section_heading])
    
    # Add Heading
    doc.add_heading(section_heading)
    
    # Add text
    for text_item in section_text:
        paragraph = doc.add_paragraph(text_item)
        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
        paragraph.paragraph_format.space_after = Pt(6)

In [11]:
# Save Document
doc.save(f'output/{filename}.docx')